### Fetch Data

In [1]:
! cp "/notebooks/storage/Manual Model.h5" "/notebooks/storage/2020-04-11(0.546).h5"

In [1]:
#! cp "/notebooks/storage/2020-04-06(0.58).h5" "/notebooks/storage/Manual Model.h5"

In [7]:
from tensorflow.keras.models import load_model
model=load_model("/storage/2020-04-06(0.58).h5",compile=False)
model.compile(optimizer="adam",loss="mean_squared_error")
print("Successfully loaded h5 model!")

import pandas as pd

df_validate = pd.read_csv("/storage/validate.csv", na_values=['NA', '?'])
df_validate['filename']="clips-"+df_validate["id"].astype(str)+".png"

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

IMAGES_DIR = "/storage/clips"

from PIL import Image
import numpy as np
from tqdm import tqdm

df = pd.DataFrame()

for i in tqdm(range(5000)):
    arr=np.expand_dims(np.array(Image.open(f"/storage/clips/clips-{25001+i}.png"))/255, 0)
    df.loc[i+25001,"clip_count"]=model.predict(arr)
df["clip_count"]=df["clip_count"].apply(lambda x: 75 if x>75 else x)
df=df.reset_index()
df.columns=["id","clip_count"]

  0%|          | 1/5000 [00:00<09:27,  8.81it/s]

Successfully loaded h5 model!


100%|██████████| 5000/5000 [04:03<00:00, 20.57it/s]


In [8]:
df.to_csv("/notebooks/submit.csv")

### Predict test data

In [48]:
from tensorflow.keras.models import load_model
model=load_model("/storage/2020-04-06(0.58).h5",compile=False)
model.compile(optimizer="adam",loss="mean_squared_error")
print("Successfully loaded h5 model!")

import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image
df=pd.DataFrame()
for i in tqdm(range(5000)):
    arr=np.expand_dims(np.array(Image.open(f"/storage/clips/clips-{45001+i}.png"))/255, 0)
    df.loc[i+45001,"clip_count"]=model.predict(arr)
df

  0%|          | 0/5000 [00:00<?, ?it/s]

Successfully loaded h5 model!


100%|██████████| 5000/5000 [05:19<00:00, 15.66it/s]


,clip_count
45001,68.247749
45002,40.110317
45003,41.237000
45004,38.798252
45005,18.474697
...,...
49996,35.309082
49997,53.150887
49998,71.147865
49999,23.043373


In [2]:
import pandas as pd
df=pd.read_csv("/notebooks/test_prediction.csv")
df.columns=["id","clip_count"]
df["clip_count"]=df["clip_count"].apply(lambda x: 75 if x>75 else x)
df["decimal"]=df["clip_count"]-df["clip_count"].astype("int")
df["integer"]=df["clip_count"].astype("int")
df["True count"]=pd.read_csv("/storage/data.csv")["clip_count"][-5000:].values
df["error"]=df["True count"]-df["clip_count"]

from sklearn.metrics import mean_squared_error
display(mean_squared_error(df["clip_count"],df["True count"]))

<Figure size 640x480 with 1 Axes>

0.5787314589089702

In [8]:
df["clip_count"]=df["after decimal"]+df["integer"]

In [18]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df["clip_count"],df["True count"])

0.5865635793156492

### Combine Three Temporary Best Results

In [14]:
import pandas as pd
df1=pd.read_csv("/notebooks/submit6.csv")
df2=pd.read_csv("/notebooks/submit7.csv")
df3=pd.read_csv("/notebooks/submit.csv")

df=pd.DataFrame()
df["id"]=df1["id"]
df["clip_count"]=(df1["clip_count"]+df2["clip_count"]+df3["clip_count"])/3
df

,id,clip_count
0,25001,2.034137
1,25002,47.229285
2,25003,63.888115
3,25004,3.408607
4,25005,44.976323
...,...,...
4995,29996,31.672199
4996,29997,54.746607
4997,29998,61.425887
4998,29999,4.965807


In [16]:
df.to_csv("/notebooks/submit8.csv")

### Global Combination

#### Get set up

In [ ]:
import pandas as pd
test=pd.read_csv("/notebooks/test.csv")
test_features=pd.read_csv("/notebooks/test_features.csv").iloc[-5000:,2:]
test["f1"]=test_features["size"].values
test["f2"]=test_features["grey_pixel_number"].values
test["f3"]=test_features["50"].values
test.drop(["True","Unnamed: 0","feature1"],axis=1,inplace=True)
test["True"]=pd.read_csv("/notebooks/storage/data.csv")[-5000:]["clip_count"].values
test.to_csv("/notebooks/test.csv")


final=pd.read_csv("/notebooks/final.csv")
submit_features=pd.read_csv("/notebooks/submit_features.csv").iloc[-5000:,2:]
final["f1"]=submit_features["size"].values
final["f2"]=submit_features["grey_pixel_number"].values
final["f3"]=submit_features["50"].values
final.to_csv("/notebooks/final.csv")

In [1]:
import os
model_list=[]
for file in os.listdir("/storage"):
    if "0.5" in file:
        model_list.append(file)

from PIL import Image
from tqdm import tqdm
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
count=7
#final=pd.DataFrame()
#final.to_csv("/notebooks/test.csv")
#final["id"]=range(45001,50001)
#final=final.set_index("id")
final=pd.read_csv("/notebooks/test.csv")
for file in model_list[6:]:
    model=load_model(f"/storage/{file}",compile=False)
    model.compile(optimizer="adam",loss="mean_squared_error")
    
    for i in tqdm(range(45001,50001)):
        arr=np.expand_dims(np.array(Image.open(f"/storage/clips/clips-{i}.png"))/255, 0)
        final.loc[i,str(count)]=model.predict(arr)[0,0]
    final[str(count)]=final[str(count)].apply(lambda x: 75 if x>75 else x)
    final.to_csv("/notebooks/test.csv")
    print(f"{count}/12 done.")
    count+=1


100%|██████████| 5000/5000 [05:11<00:00, 16.07it/s]


7/12 done.


100%|██████████| 5000/5000 [04:51<00:00, 17.14it/s]


8/12 done.


100%|██████████| 5000/5000 [04:47<00:00, 17.41it/s]


...

10/12 done.


100%|██████████| 5000/5000 [04:49<00:00, 17.26it/s]  


11/12 done.


100%|██████████| 5000/5000 [04:50<00:00, 17.20it/s]  


12/12 done.


#### Test set for XGB

In [98]:
import pandas as pd
test=pd.read_csv("/notebooks/test.csv")

import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(test.iloc[:4500,2:-1].values,label=test.iloc[:4500,-1].values)
dtest = xgb.DMatrix(test.iloc[4500:,2:-1].values,label=test.iloc[4500:,-1].values)
param = {'max_depth':10, 'eta':0.5, 'objective':'reg:squarederror',"silent":1}
evallist = [(dtest,'eval'),(dtrain,'train')]

model=xgb.train(param, dtrain,200,evallist) 

[0]	eval-rmse:22.16201	train-rmse:21.55981
[1]	eval-rmse:11.12824	train-rmse:10.85018
[2]	eval-rmse:5.65274	train-rmse:5.49184
[3]	eval-rmse:2.96067	train-rmse:2.83610
[4]	eval-rmse:1.68070	train-rmse:1.53851
[5]	eval-rmse:1.12956	train-rmse:0.91251
[6]	eval-rmse:0.93938	train-rmse:0.62276
[7]	eval-rmse:0.88437	train-rmse:0.49823
[8]	eval-rmse:0.86733	train-rmse:0.43782
[9]	eval-rmse:0.86369	train-rmse:0.41141
[10]	eval-rmse:0.86133	train-rmse:0.39061
[11]	eval-rmse:0.86499	train-rmse:0.37599
[12]	eval-rmse:0.86457	train-rmse:0.36173
[13]	eval-rmse:0.86294	train-rmse:0.35311
[14]	eval-rmse:0.86405	train-rmse:0.34546
[15]	eval-rmse:0.86110	train-rmse:0.32487
[16]	eval-rmse:0.86018	train-rmse:0.32259
[17]	eval-rmse:0.85952	train-rmse:0.31043
[18]	eval-rmse:0.86256	train-rmse:0.30554
[19]	eval-rmse:0.86352	train-rmse:0.28191
[20]	eval-rmse:0.86547	train-rmse:0.27337
[21]	eval-rmse:0.86340	train-rmse:0.25556
[22]	eval-rmse:0.86001	train-rmse:0.25161
[23]	eval-rmse:0.85477	train-rmse:0.2405

#### Train Dense Neural Network

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(test.iloc[:,:-1],
                                                  test.iloc[:,-1],
                                                  test_size=0.1)
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
model=Sequential()
model.add(Dense(12,input_dim=12,activation="relu"))
model.add(Dense(6,activation="relu"))
#model.add(Dense(3,activation="relu"))
model.add(Dense(1,activation="relu"))
model.compile(loss="mse",optimizer="adam")
monitor=EarlyStopping(monitor='val_loss',min_delta=1e-3, 
        patience=20, verbose=0, mode='auto', restore_best_weights=True)
model.fit(X_train.values,y_train.values,validation_data=(X_test.values,y_test.values),
          epochs=200,callbacks=[monitor],verbose=2)
print("Done")

from sklearn.metrics import mean_squared_error
print(mean_squared_error(model.predict(X_test.values),y_test),
mean_squared_error(X_test.values.mean(axis=1),y_test))

model.save("/notebooks/Prediction.h5")

Done


In [ ]:
import pandas as pd
df=pd.read_csv("/notebooks/final.csv")
arr=model.predict(df.iloc[:,1:].values)
submit=pd.DataFrame({"id":range(25001,30001),"clip_count":arr.flatten()})
submit.to_csv("/notebooks/submit10.csv")

df["clip_count"]=df.iloc[:,1:].apply(lambda x: x.mean(),axis=1)
df=df.drop(df.columns[1:13],axis=1)
df.to_csv("/notebooks/submit9.csv")

#### Ordinary Linear Model

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd


"""MSE=[]
for i in range(100):
    X_train, X_test, y_train, y_test=train_test_split(test.iloc[:,:-1],
                                                      test.iloc[:,-1],
                                                      test_size=0.2)

    model=LinearRegression().fit(X_train, y_train)
    pred=model.predict(X_test)
    MSE.append(mean_squared_error(pred,y_test.values))
    print(MSE[-1])
    np.array(MSE).mean()"""

test=pd.read_csv("/notebooks/test.csv")
model=LinearRegression().fit(test.drop(["id","TRUE"],axis=1),test["TRUE"])
df=pd.read_csv("/notebooks/final.csv")
df["clip_count"]=model.predict(df.iloc[:,1:])
df[["id","clip_count"]].to_csv("/notebooks/submit_LRG.csv")
df

,id,1,2,3,4,5,6,7,8,9,...,f1,f2,f3,13,14,15,16,17,18,clip_count
0,25001,2.078502,2.048270,2.176548,2.181710,2.171663,2.042809,2.054140,2.218273,1.979649,...,2868,499,65050,2.013689,2.107004,2.010603,2.404801,2.069614,2.198115,2.072007
1,25002,47.102856,47.471851,47.402054,47.428837,46.916428,46.533054,47.178307,46.916542,46.560528,...,47019,10977,54836,47.192917,46.937836,47.383320,47.509872,48.650318,48.066029,47.003592
2,25003,63.601151,63.744083,63.498966,64.263016,63.850052,63.831371,63.330708,63.916481,64.481827,...,55200,12753,53104,63.066990,63.650635,63.486897,63.994404,63.896095,63.706322,64.066205
3,25004,3.402958,3.249214,3.299937,3.176938,3.356996,3.380183,3.544874,3.205535,3.324197,...,3705,627,64924,3.014899,3.257248,3.233060,3.069120,3.185067,3.282437,3.334631
4,25005,44.893665,45.110535,44.781345,44.324539,44.479763,44.815819,44.842922,44.496338,44.447712,...,45646,10110,55722,44.922932,45.427040,45.568714,43.645378,44.773441,45.163376,44.903638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29996,31.803255,31.420254,31.000479,31.894354,32.291943,31.517700,31.811995,31.809288,31.404495,...,34695,7480,58270,30.859953,30.938112,32.053539,31.498846,31.262012,32.050858,31.744798
4996,29997,55.309746,53.856491,55.245605,55.156731,55.386330,55.594093,55.457638,55.631950,55.001411,...,55150,12209,53677,53.447701,54.243759,54.131619,55.441936,54.554523,54.286480,54.389249
4997,29998,61.431679,61.042538,61.438362,61.994614,61.362617,62.571304,61.454952,61.281811,61.259480,...,54106,12456,53412,61.059872,60.931667,61.399853,61.751053,62.483074,61.677982,61.683299
4998,29999,4.966510,4.951837,4.872506,4.947905,4.836755,4.822618,4.945584,4.762985,4.751533,...,7211,1425,64164,4.935795,4.967635,5.335175,4.839327,4.896451,4.940805,4.951068


In [5]:
test

,id,1,2,3,4,5,6,7,8,9,...,f1,f2,f3,TRUE,13,14,15,16,17,18
0,45001,66.483871,68.247749,66.648193,67.212799,66.653748,67.263458,67.051575,67.126739,66.983757,...,61355,14280,51635,68,67.068939,67.058807,68.732140,66.852257,67.853073,68.737343
1,45002,40.468941,40.110317,40.656063,40.764927,40.876328,40.335373,40.375340,40.574928,40.583611,...,40761,9209,56548,40,40.513443,40.126709,40.082539,40.168465,40.051807,40.884926
2,45003,41.460964,41.237000,41.067108,41.374786,41.528965,41.465656,41.205536,41.467598,41.903027,...,45467,9829,55936,41,40.529606,41.791656,42.395710,41.541862,41.879608,42.029758
3,45004,38.751732,38.798252,38.221867,38.272499,38.189594,38.107555,38.276199,37.992920,38.329498,...,37876,8451,57317,39,38.873005,38.273144,39.530891,38.209827,37.762806,38.366428
4,45005,17.631557,18.474697,17.742939,17.419392,17.760508,17.705917,17.507004,17.828539,17.556057,...,18560,3932,61734,19,18.065754,18.086224,18.998547,17.480804,17.940041,18.797537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,49996,35.934433,35.309082,35.609249,35.316940,35.871963,35.322407,35.144230,35.696674,35.814716,...,35775,7886,57833,35,35.250870,35.313862,35.307575,35.863522,35.563389,36.225704
4996,49997,54.239372,53.150887,53.655266,54.484295,53.709156,54.309490,53.896324,54.291805,54.020184,...,50962,11382,54432,54,53.494812,53.149113,53.200169,53.131248,54.002472,54.264385
4997,49998,71.145584,71.147865,70.953239,71.699875,71.459007,70.983078,70.717415,71.407021,71.009995,...,63938,15244,50704,72,70.673981,70.489418,71.557632,72.032745,71.640343,71.700966
4998,49999,23.010813,23.043373,22.641993,22.648394,22.773958,22.792301,22.843092,22.701715,22.557844,...,24989,5269,60416,24,22.766565,22.996029,23.217543,22.926369,23.010355,23.527931


#### Majority Vote

In [31]:
import pandas as pd
import numpy as np
from scipy.stats import mode
df=pd.read_csv("/notebooks/final.csv")
#df["True"]=pd.read_csv("/notebooks/storage/data.csv")[-5000:]["clip_count"].values
df.iloc[:,1:]=df.iloc[:,1:].applymap(lambda x: np.round(x))
df["clip_count"]=df.iloc[:,1:].apply(lambda x:mode(x)[0][0],axis=1)
df.iloc[:,[0,-1]].to_csv("/notebooks/submit_MJV.csv")
df
#This is for test validation
#from sklearn.metrics import mean_squared_error
#mean_squared_error(df["clip_count"].values,df["True"].values)

,id,1,2,3,4,5,6,7,8,9,10,11,12,clip_count
0,25001.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,25002.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
2,25003.0,64.0,64.0,63.0,64.0,64.0,64.0,63.0,64.0,64.0,65.0,63.0,64.0,64.0
3,25004.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0
4,25005.0,45.0,45.0,45.0,44.0,44.0,45.0,45.0,44.0,44.0,45.0,44.0,45.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29996.0,32.0,31.0,31.0,32.0,32.0,32.0,32.0,32.0,31.0,32.0,32.0,32.0,32.0
4996,29997.0,55.0,54.0,55.0,55.0,55.0,56.0,55.0,56.0,55.0,55.0,55.0,55.0,55.0
4997,29998.0,61.0,61.0,61.0,62.0,61.0,63.0,61.0,61.0,61.0,62.0,62.0,62.0,61.0
4998,29999.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


#### AutoML

In [62]:
import pandas as pd

test=pd.read_csv("/notebooks/test.csv")
test.set_index("id",inplace=True)
test["True"]=pd.read_csv("/notebooks/storage/data.csv")[-5000:]["clip_count"].values

import h2o
from h2o.automl import H2OAutoML
h2o.init()
import random
train=test.iloc[random.sample(range(5000),4000),:]
mask=test.index.isin(train.index)
test=test.loc[(1-mask).astype(bool),:]
x=train.columns[:-1]
y=train.columns[-1]

aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)
preds = aml.predict(test)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...


H2OStartupError: Cannot find Java. Please install the latest JRE from
http://www.oracle.com/technetwork/java/javase/downloads/index.html

#### Global Average

In [17]:
import pandas as pd
df=pd.read_csv("/notebooks/final.csv")
df["clip_count"]=df.iloc[:,1:].apply(lambda x:x.mean(),axis=1)
df[["id","clip_count"]].to_csv("/notebooks/submit_GAG.csv")

#### Random Forest 

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(test.iloc[:,2:-1],
                                                  test.iloc[:,-1],
                                                  test_size=0.1)

rf=RandomForestClassifier(n_estimators=5000,max_depth=5,verbose=1,n_jobs=8)
rf.fit(X_train,y_train)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    5.1s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    7.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:   10.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:   13.2s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:   16.7s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:   20.6s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:   20.7s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5000,
                       n_jobs=8, oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

In [53]:
pred=rf.predict(X_test)
from sklearn.metrics import mean_squared_error
display(mean_squared_error(pred,y_test))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    3.0s finished


0.718

### Get 12 Results

In [1]:
import os
model_list=[]
for file in os.listdir("/storage"):
    if "0.6" in file or "0.7" in file or "0.8" in file:
        model_list.append(file)

from PIL import Image
from tqdm import tqdm
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
count=16
final=pd.read_csv("/notebooks/final.csv")
test=pd.read_csv("/notebooks/test.csv")
for file in model_list[3:]:
    model=load_model(f"/storage/{file}",compile=False)
    model.compile(optimizer="adam",loss="mean_squared_error")
    
    for i in tqdm(range(25001,30001)):
        arr=np.expand_dims(np.array(Image.open(f"/storage/clips/clips-{i}.png"))/255, 0)
        final.loc[i,str(count)]=model.predict(arr)[0,0]
    
    for i in tqdm(range(45001,50001)):
        arr=np.expand_dims(np.array(Image.open(f"/storage/clips/clips-{i}.png"))/255, 0)
        test.loc[i,str(count)]=model.predict(arr)[0,0]
        
    final[str(count)]=final[str(count)].apply(lambda x: 75 if x>75 else x)
    test[str(count)]=test[str(count)].apply(lambda x: 75 if x>75 else x)

    final.to_csv("/notebooks/final.csv")
    test.to_csv("/notebooks/test.csv")
    print(f"{count} done.")
    count+=1


100%|██████████| 5000/5000 [04:05<00:00, 20.35it/s]


16 done.


100%|██████████| 5000/5000 [03:43<00:00, 22.39it/s]


17 done.


100%|██████████| 5000/5000 [03:50<00:00, 21.68it/s]  


18 done.


 31%|███       | 1541/5000 [01:07<02:31, 22.87it/s]


KeyboardInterrupt: 